In [2]:
import logging
import time

from celery import shared_task

from asynclog import AsyncLogDispatcher

In [3]:
def write_record(record):
    time.sleep(1)
    
class SyncLogHandler(logging.Handler):
    def emit(self, record):
        time.sleep(1)

### AsyncLogDispatcher (use thread)

In [3]:
async_logger = logging.getLogger('Async Logger')
async_logger.setLevel(logging.INFO)

In [4]:
async_handler = AsyncLogDispatcher(write_record)
async_handler.setLevel(logging.INFO)
async_logger.addHandler(async_handler)

In [5]:
async_logger.info('Test log')

In [6]:
%timeit async_logger.info('Test log')

40.5 µs ± 386 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### SyncLogHandler

In [7]:
sync_logger = logging.getLogger('Sync Logger')
sync_logger.setLevel(logging.INFO)

sync_handler = SyncLogHandler()
sync_handler.setLevel(logging.INFO)
sync_logger.addHandler(sync_handler)

In [8]:
sync_logger.info('Test log')

In [9]:
%timeit sync_logger.info('Test log')

1 s ± 1.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### AsyncLogDispatcher (use celery)

In [4]:
from unittest import mock

In [5]:
from asynclog.tests.test_handler import app, write_task, has_celery

celery_logger = logging.getLogger('Celery logger')
celery_logger.setLevel(logging.INFO)

if not has_celery:
    write_task.delay = mock.MagicMock()
celery_handler = AsyncLogDispatcher(write_task, use_thread=False, use_celery=True)
celery_handler.setLevel(logging.INFO)
celery_logger.addHandler(celery_handler)

In [12]:
celery_logger.info('Test log')

In [13]:
%timeit celery_logger.info('Test log')

857 µs ± 71.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
